# Laboratoire 3: Optimisation

## 1. Fonctions d'optimisation

Dans cette section, vous testerez différentes fonctions d'optimisation et observerez leurs effets sur l'entraînement.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score
from deeplib.history import History
from deeplib.datasets import train_valid_loaders, load_cifar10, load_mnist

from torch.autograd import Variable
from torchvision.transforms import ToTensor

from deeplib.net import CifarNet, MnistNet

from deeplib.training import train, validate_ranking, test
from deeplib.visualization import show_worst, show_random, show_best

cifar_train, cifar_test = load_cifar10()

Voici un exemple d'entraînement avec SGD.

In [ ]:
batch_size = 128
lr = 0.01
n_epoch = 10

In [ ]:
model = CifarNet()
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr)
history_sgd = train(model, optimizer, cifar_train, n_epoch, batch_size)
history_sgd.display_accuracy()
history_sgd.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))

## Exercice

Comparez trois différentes stratégies d'optimisation:
1. [SGD](http://pytorch.org/docs/master/optim.html#torch.optim.SGD)
2. SGD + Momentum accéléré de Nesterov
3. [Adam](http://pytorch.org/docs/master/optim.html#torch.optim.Adam) 

Complétez cette cellule pour entraîner avec SGD + Momentum accéléré de Nesterov. Utilisez un momentum de 0.9.

In [ ]:
model = CifarNet()
model.cuda()
# optimizer = 
history_SGDMN = train(model, optimizer, cifar_train, n_epoch, batch_size)
history_SGDMN.display_accuracy()
history_SGDMN.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))

Complétez cette cellule pour entraîner avec Adam

In [ ]:
model = CifarNet()
model.cuda()
# optimizer = 
history_adam = train(model, optimizer, cifar_train, n_epoch, batch_size)
history_adam.display_accuracy()
history_adam.display_loss()
print('Précision en test: {:.2f}'.format(test(model, cifar_test, batch_size)))

Quelle méthode semble être la meilleure dans ce cas-ci?
Remarquez-vous une différence d'overfitting?

## 2. Horaire d'entraînement

Une pratique courante utilisé en deep learning est de faire diminuer le learning rate pendant l'entraînement.

Pour ce faire PyTorch fourni plusieurs fonctions (ExponentialLR, LambdaLR, MultiStepLR, etc.)

Voici un exemple avec ExponentialLR.

In [ ]:
model = CifarNet()
model.cuda()

batch_size = 128
lr = 0.01
n_epoch = 10

optimizer = optim.SGD(model.parameters(), lr=lr)

gamma = 0.99
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)

history = train(model, optimizer, cifar_train, n_epoch, batch_size, scheduler=scheduler, use_gpu=True)

In [ ]:
history.display_lr()

## Exercice

Utilisez [MultiStepLR](http://pytorch.org/docs/master/optim.html#torch.optim.lr_scheduler.MultiStepLR) pour modifier le learning rate un epoch précis. 

1. Commencez avec un learning rate trop élevé pour que le réseau puisse apprendre quelque chose.
2. Diminuez le progressivement jusqu'à ce que le réseau apprenne.
3. Trouvez le moment où la validation semble avoir atteint un plateau.
4. Diminuez le learning par 2 à ce moment et réentraîner le réseau.

In [ ]:
torch.manual_seed(42)
model = CifarNet()
model.cuda()

epoch_list = []

batch_size = 128
lr = 10
n_epoch = 20

optimizer = optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=epoch_list, gamma=0.5)

history = train(model, optimizer, cifar_train, n_epoch, batch_size, scheduler=scheduler, use_gpu=True)

In [ ]:
history.display_lr()
history.display_loss()
history.display_accuracy()

Voyez-vous une différence en diminuant le learning rate par 2 après x epochs?<br>
Pourquoi?

## 3. Batch Normalization

Voici l'architecture du réseau de neurones convolutionnels que vous avez utilisé jusqu'à présent pour faire de la classification sur Cifar10.

In [ ]:
import torch.nn.functional as F

class CifarNetBatchNorm(nn.Module):
    def __init__(self):
        super(CifarNetBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, padding=1)
        self.conv2 = nn.Conv2d(10, 50, 3, padding=1)
        self.conv3 = nn.Conv2d(50, 150, 3, padding=1)
        self.fc1 = nn.Linear(150 * 8 * 8, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        return x

    @staticmethod
    def num_flat_features(x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

## Exercice

Modifier l'architecture du réseau en ajoutant de la batch norm entre les couches de convolutions et entraîner le nouveau réseau.

Comparer l'entraînement du réseau avec et sans la batch norm (Section 1 avec SGD).<br>
Que remarquez-vous?<br>

In [ ]:
model = CifarNetBatchNorm()
model.cuda()

lr = 0.01
batch_size = 128
n_epoch = 10

optimizer = optim.SGD(model.parameters(), lr=lr)

history = train(model, optimizer, cifar_train, n_epoch, batch_size, use_gpu=True)
history.display_accuracy()
history.display_loss()

### Effet de la batch norm sur le learning rate

Commençons par entraîner un réseau avec un haut learning rate.

In [ ]:
lr = 0.5
batch_size = 1024
n_epoch = 10

In [ ]:
model = CifarNet()
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr)

history = train(model, optimizer, cifar_train, n_epoch, batch_size, use_gpu=True)

Essayons maintenant d'entraîner le réseau utilisant la batchnorm avec les mêmes hyperparamètres.

In [ ]:
model = CifarNetBatchNorm()
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr)

history = train(model, optimizer, cifar_train, n_epoch, batch_size, use_gpu=True)

Que pouvez-vous conclure sur l'effet de la batch norm sur le learning rate?

## Analyse

Après l'entraînement, il est important d'analyser les résultats obtenus.
Commençons par tester le réseau en utilisant la fonction `validate_ranking`.
Cette fonction sépare les résultats bien classés des erreurs et retourne pour chaque image, un score (qu'on peut voir comme une probabilité), la vraie classe et la classe prédite.

In [ ]:
cifar_test.transform = ToTensor()
loader, _ = train_valid_loaders(cifar_test, batch_size, train_split=1)
good, errors = validate_ranking(model, loader, use_gpu=True)

Maintenant, regardons quelques exemples d'images bien classées.

In [ ]:
show_random(good)

Et quelques exemples mal classées.

In [ ]:
show_random(errors)

Il est aussi possible de regarder les exemples où le réseau est le plus confiant.

In [ ]:
show_best(good)

Ou l'inverse, ceux qui ont obtenus les moins bons scores.

In [ ]:
show_worst(errors)

Finalement, il peut être intéressant de regarder les exemples les plus difficiles.
Soit ceux qui ont été bien classé, mais qui ont eu un mauvais score.

In [ ]:
show_worst(good)

Ou ceux qui été mal classé, mais qui ont quand même réussi à obtenir un bon score.

In [ ]:
show_best(errors)

En observant les résultats obtenus, que pouvez-vous dire sur les performances du réseau? <br>
Quelle classe semble être facile? Pourquoi? <br>
Quelle classe semble être difficile? Pourquoi? <br>